In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import os
import sys
sys.path.append("../")
from dynaprot.evaluation.visualizer import plot_3d_gaussian_ellipsoids,plot_3d_gaussian_comparison
from openfold.utils.rigid_utils import  Rigid
from tqdm import tqdm
import plotly.express as px

config_dir = "../configs"


In [2]:
device = 1

In [15]:
import yaml
from pathlib import Path

with open(config_dir+"/data/atlas_config.yaml", "r") as file:
    dataconfig = yaml.safe_load(file)
    
with open(config_dir+"/model/dynaprot_simple.yaml", "r") as file:
    modelconfig = yaml.safe_load(file)
    
modelconfig["data_config"] = dataconfig
    
print(dataconfig["data_dir"])

from dynaprot.data.datasets import DynaProtDataset, OpenFoldBatchCollator

dataset = DynaProtDataset(dataconfig, split="test")
print(len(dataset))
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        collate_fn=OpenFoldBatchCollator(),
        num_workers=12,
        shuffle=False,
    )


/data/cb/scratch/datasets/atlas_dynamics_labels_calpha
82


In [20]:
from dynaprot.model.architecture import DynaProt

# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-126/DYNAMICS-126/checkpoints/step13112.ckpt", cfg=modelconfig).to(device)
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-126/DYNAMICS-126/checkpoints/step=174798.ckpt", cfg=modelconfig).to(device).eval()
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-134/DYNAMICS-134/checkpoints/step=30000.ckpt", cfg=modelconfig).to(device).eval()
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-135/DYNAMICS-135/checkpoints/step=4017.ckpt", cfg=modelconfig).to(device).eval()
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-135/DYNAMICS-135/checkpoints/step=31000.ckpt", cfg=modelconfig).to(device).eval()
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-154/DYNAMICS-154/checkpoints/step=26143.ckpt", cfg=modelconfig).to(device).eval()     # Best to date (rmwd 1.04) with centroids, bures distance loss, atlas no replicates?
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-167/DYNAMICS-167/checkpoints/epoch=2400-step=31213.ckpt", cfg=modelconfig).to(device).eval()                 # Best to date with centroids, bures distance loss, atlas with replicates
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-150/DYNAMICS-150/checkpoints/step=47060.ckpt",cfg=modelconfig).to(device).eval()        # Centroids, MSE Loss, atlas no replicates

# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-183/DYNAMICS-183/checkpoints/epoch=2140-step=27833.ckpt",cfg=modelconfig).to(device).eval()     # Best to date (rmwd 0.84/0.97 on train/test) Calphas, Bures distance Loss, atlas no replicates but with augmentation
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-183/DYNAMICS-183/checkpoints/step=3000.ckpt",cfg=modelconfig).to(device).eval()                 # Earlier checkpoint Calphas, Bures distance Loss, atlas no replicates but with augmentation 

# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-247/DYNAMICS-247/checkpoints/step=1300.ckpt",cfg=modelconfig).to(device).eval()                     #(rmwd 0.90/1.21/0.95 on train/val/test) cholesky, bures distance, ddp4
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-248/DYNAMICS-248/checkpoints/step=1328.ckpt",cfg=modelconfig).to(device).eval()                     # (rmwd 1.36/1.46/1.30 on train/val/test) cholesky, mse, ddp4
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-271/DYNAMICS-271/checkpoints/step=3000.ckpt",cfg=modelconfig).to(device).eval()                     # direct, mse, ddp4
model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-252/DYNAMICS-252/checkpoints/step=12000.ckpt",cfg=modelconfig).to(device).eval()                     #   (rmwd 0.902/1.05/0.944 on train/val/test) cholesky, mse, ddp4
# model = DynaProt.load_from_checkpoint("../.neptune/DYNAMICS-258/DYNAMICS-258/checkpoints/step=12000.ckpt",cfg=modelconfig).to(device).eval()                     # (rmwd 0.895/1.28/1.055 on train/val/test)

model

DynaProt(
  (sequence_embedding): Embedding(21, 128)
  (ipa_blocks): ModuleList(
    (0-7): 8 x InvariantPointAttention(
      (linear_q): Linear(in_features=128, out_features=64, bias=True)
      (linear_kv): Linear(in_features=128, out_features=128, bias=True)
      (linear_q_points): Linear(in_features=128, out_features=48, bias=True)
      (linear_kv_points): Linear(in_features=128, out_features=144, bias=True)
      (linear_b): Linear(in_features=128, out_features=4, bias=True)
      (linear_out): Linear(in_features=704, out_features=128, bias=True)
      (softmax): Softmax(dim=-1)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (covars_predictor): Linear(in_features=128, out_features=6, bias=True)
  (loss): DynaProtLoss()
)

## Test RMWD variance contribution (bures distance) 

In [19]:
from dynaprot.evaluation import metrics

rmwds = []
for prot in tqdm(dataloader):
    pred = model(prot["aatype"].argmax(dim=-1).to(device), Rigid.from_tensor_4x4(prot["frames"].to(device)), prot["resi_pad_mask"].to(device))
    mask = prot["resi_pad_mask"].bool().to(device)
    true_covars = prot["dynamics_covars"].to(device).float()[mask]
    predicted_covars =  pred["covars"][mask]

    rmwds.append(torch.sqrt(metrics.bures_distance(predicted_covars,true_covars)).item())

px.box(rmwds)

100%|██████████| 82/82 [00:05<00:00, 13.68it/s]


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from dynaprot.model.architecture import DynaProt
from dynaprot.data.datasets import DynaProtDataset, OpenFoldBatchCollator
from dynaprot.evaluation import metrics
from openfold.utils.rigid_utils import Rigid  # Assuming you use OpenFold

def eval_checkpoint(ckpt_path, modelconfig, dataconfig, metric, device="cuda"):
    """Evaluates a checkpoint and appends all individual results to a CSV file."""
    
    ckpt_name = ckpt_path.split("/")[-1]
    csv_path = f"./benchmarking/evals/dynaprot_{metric}.csv"
    
    try:
        eval_df = pd.read_csv(csv_path)
    except FileNotFoundError:
        eval_df = pd.DataFrame(columns=["checkpoint", "split", "protein", "value"])

    if ckpt_name in eval_df["checkpoint"].values:
        print(f"Skipping {ckpt_name}, already in CSV.")
        return

    print(f"Evaluating {ckpt_name}...")
    model = DynaProt.load_from_checkpoint(ckpt_path, cfg=modelconfig).to(device).eval()
    
    new_data = []
    
    for split in ["train", "val", "test"]:
        dataset = DynaProtDataset(dataconfig, split=split)

        for i in tqdm(range(len(dataset)), desc=f"Processing {split} split"):
            prot = dataset[i]  
            protein_name = dataset.protein_list[i]  

            pred = model(
                prot["aatype"].argmax(dim=-1).to(device).unsqueeze(0),
                Rigid.from_tensor_4x4(prot["frames"].to(device)).unsqueeze(0),
                prot["resi_pad_mask"].to(device).unsqueeze(0)
            )
            mask = prot["resi_pad_mask"].bool().to(device).unsqueeze(0)
            true_covars = prot["dynamics_covars"].to(device).float().unsqueeze(0)[mask]
            predicted_covars = pred["covars"][mask]
            rmwd = torch.sqrt(metrics.bures_distance(predicted_covars, true_covars)).item()

            new_data.append({
                "checkpoint": ckpt_name,
                "split": split,
                "protein": protein_name,  
                "value": rmwd
            })

    new_df = pd.DataFrame(new_data)
    eval_df = pd.concat([eval_df, new_df], ignore_index=True)
    eval_df.to_csv(csv_path, index=False)

    print(f"Saved results for {ckpt_name} in {csv_path}")
    
eval_checkpoint(ckpt_path="../ckpt/cholesky_bures_dd4-step=000500.ckpt",modelconfig=modelconfig,dataconfig=dataconfig, metric="rmwd")


Evaluating cholesky_bures_dd4-step=000500.ckpt...


Processing test split: 100%|██████████| 82/82 [00:14<00:00,  5.85it/s]


Saved results for cholesky_bures_dd4-step=000500.ckpt in ./benchmarking/evals/dynaprot_rmwd.csv


### RMWD Results

In [6]:
import pandas as pd
import plotly.express as px

csv_path = "./benchmarking/evals/dynaprot_rmwd.csv"
rmwd_evals = pd.read_csv(csv_path)
rmwd_evals



,checkpoint,split,protein_id,protein,value
0,cholesky_bures_dd4-step=000500.ckpt,train,NaN,1k5n_A,4.084709
1,cholesky_bures_dd4-step=000500.ckpt,train,NaN,5e3e_A,0.923344
2,cholesky_bures_dd4-step=000500.ckpt,train,NaN,3ilw_A,0.683373
3,cholesky_bures_dd4-step=000500.ckpt,train,NaN,3bl9_A,3.311295
4,cholesky_bures_dd4-step=000500.ckpt,train,NaN,1zd7_B,0.553672
...,...,...,...,...,...
37444,cholesky_bures_dd4-step=014000.ckpt,test,NaN,6q10_A,0.922165
37445,cholesky_bures_dd4-step=014000.ckpt,test,NaN,6jv8_A,0.818938
37446,cholesky_bures_dd4-step=014000.ckpt,test,NaN,6lrd_A,1.175051
37447,cholesky_bures_dd4-step=014000.ckpt,test,NaN,6tly_A,2.096895


### Predicting Cholesky Factor with Bures Distance across all checkpoints (train,val,test)

In [ ]:
model_name = "cholesky_bures"
df_filtered = rmwd_evals[rmwd_evals["checkpoint"].str.contains(model_name, na=False)]
df_filtered["step"] = df_filtered["checkpoint"].str.extract(r"step=(\d+)").astype(int)


fig = px.box(
    df_filtered,
    x="step",  
    y="value",  
    color="split", 
    title=f"Distribution of RMWD Across Checkpoints {model_name}",
    labels={"value": "RMWD Value", "checkpoint": "Checkpoint"},
    boxmode="group" 
)

fig.show()

df_median = df_filtered.groupby(["step", "split"])["value"].median().reset_index()
fig = px.line(
    df_median,
    x="step",
    y="value",
    color="split",
    title="Median RMWD Across Checkpoints",
    labels={"value": "Median RMWD Value", "step": "Step"},
    markers=True  # Adds points to lines for better readability
)

fig.show()

/tmp/ipykernel_1048986/2503424914.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Predicting Cov Directly with MSE Distance across all checkpoints (train,val,test)

In [ ]:
model_name = "direct_mse"
df_filtered = rmwd_evals[rmwd_evals["checkpoint"].str.contains(model_name, na=False)]
df_filtered["step"] = df_filtered["checkpoint"].str.extract(r"step=(\d+)").astype(int)


fig = px.box(
    df_filtered,
    x="step",  
    y="value",  
    color="split", 
    title=f"Distribution of RMWD Across Checkpoints {model_name}",
    labels={"value": "RMWD Value", "checkpoint": "Checkpoint"},
    boxmode="group" 
)

fig.show()

df_median = df_filtered.groupby(["step", "split"])["value"].median().reset_index()
fig = px.line(
    df_median,
    x="step",
    y="value",
    color="split",
    title="Median RMWD Across Checkpoints",
    labels={"value": "Median RMWD Value", "step": "Step"},
    markers=True
)

fig.show()

/tmp/ipykernel_1048986/2365364250.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
rmwd_evals["step"] = rmwd_evals["checkpoint"].str.extract(r"step=(\d+)").astype(int)
rmwd_evals["model_type"] = rmwd_evals["checkpoint"].str.split("-step").str[0]
df_test = rmwd_evals[rmwd_evals["split"] == "test"].copy()
df_median_test = df_test.groupby(["step", "model_type"])["value"].median().reset_index()
fig = px.line(
    df_median_test,
    x="step",
    y="value",
    color="model_type",
    title="Median RMWD for Test Split Across Model Types",
    labels={"value": "Median RMWD Value", "step": "Step", "model_type": "Model Type"},
    markers=True  # Adds points to lines for better readability
)

fig.show()

In [13]:
import mdtraj

# def compute_gaussians_per_residue(traj):
#     num_residues = traj.topology.n_residues
#     means = np.zeros((num_residues, 3))       # Shape (n_residues, 3) for (x, y, z)
#     covariances = np.zeros((num_residues, 3,3))   # Shape (n_residues, 3) for (x, y, z)

#     for i, residue in enumerate(traj.topology.residues):

#         atom_indices = [atom.index for atom in residue.atoms]
        
#         # Extract xyz coordinates for all atoms in the residue across all frames
#         # scale nanometers to angstroms (x10)
#         xyz = np.mean(traj.xyz[:, atom_indices, :],axis=1) # shape (n_frames, 3)  frames by residue i's position (mean pos of atoms) 

#         # Compute mean and variance across all frames for each atom
#         means[i] = np.mean(xyz, axis=0)  # shape (1, 3)

#         centered_data = xyz - means[i]
        
#         covariances[i] = centered_data.T @ centered_data /(centered_data.shape[0] - 1)  # shape (3, 3) 
        
#     return torch.from_numpy(covariances)

def compute_gaussians_per_residue(traj, calpha = True):
    num_residues = traj.topology.n_residues
    means = np.zeros((num_residues, 3),dtype=np.float64)       # Shape (n_residues, 3) for (x, y, z)
    covariances = np.zeros((num_residues, 3,3),dtype=np.float64)   # Shape (n_residues, 3) for (x, y, z)
    residuecoords = []

    for i, residue in enumerate(traj.topology.residues):
        use_calpha = calpha
        if use_calpha:
            ca_atom = [atom.index for atom in residue.atoms if atom.name == 'CA']
            if ca_atom:
                xyz = traj.xyz[:, ca_atom[0], :].astype(np.float64) * 10           # shape (T, 3)
            else:
                use_calpha = False  # calpha wasnt found
        
        if not use_calpha:
            atom_indices = [atom.index for atom in residue.atoms]   # Extract xyz coordinates for all atoms in the residue across all frames
            # scale nanometers to angstroms (x10)
            xyz = np.mean(traj.xyz[:, atom_indices, :].astype(np.float64),axis=1) * 10 # shape (T, 3)  frames by residue i's position (mean pos of atoms) 

        mean_xyz = np.mean(xyz, axis=0).astype(np.float64)  # shape (1, 3)
        centered_xyz = xyz-mean_xyz # shape (T,3)
        residuecoords.append(centered_xyz)

        # Compute mean and variance across all frames for each residue
        means[i] = mean_xyz
        covariances[i] = (centered_xyz.T @ centered_xyz /(centered_xyz.shape[0] - 1)).astype(np.float64)  # shape (3, 3) 
    
    return  torch.from_numpy(covariances)

        
path = "/data/cb/scratch/datasets/alphaflow_ensembles/alphaflow_md_templates_base_202402"
ref_path = "/data/cb/scratch/datasets/atlas/"
af_rmwds = []

for prot in tqdm(np.load("../dynaprot/data/preprocessing/protein_lists/atlas_chains_test.npy")):
    traj = mdtraj.load_pdb(os.path.join(path,f"{prot}.pdb"))
    # ref = mdtraj.load_pdb(os.path.join(ref_path,prot,f"{prot}.pdb"))     
    # traj.superpose(ref) 
    traj.superpose(traj,frame=0)
    af_covs = compute_gaussians_per_residue(traj)
    gt = torch.load(f"/data/cb/scratch/datasets/atlas_dynamics_labels_replicates/{prot}/{prot}.pt")
    gt_covs = covars = gt["dynamics_covars"]
    af_rmwds.append(torch.sqrt(metrics.bures_distance(af_covs,gt_covs)).item())



100%|██████████| 82/82 [13:22<00:00,  9.79s/it]


In [14]:
px.box(af_rmwds)